In [1]:
pip install fastapi

In [1]:
from fastapi import FastAPI

In [2]:
#from pymongo import MongoClient
import pandas as pd
from typing import Union
import json

In [3]:
df = pd.read_csv("ACM_IEEE_ScDir_Finale_Data.csv")

In [4]:
del df['Unnamed: 0']
df.head(1)


,Document Title,Authors,Universities,Citations,Affiliations,Publisher,DOI,Publication Year
0,Blockchain adoption and channel selection stra...,"['Bengang Gong', 'Huaimiao Zhang', 'Yiling Gao...",['Anhui Polytechnic University No8 Beijing Zho...,0.0,China,Science direct,https://www.sciencedirect.com/science/article/...,2022


In [5]:
# client = MongoClient('localhost',27017)  # 27017 is the default port number for mongodb
# db=client["myData"]
# scFinal=db["dataFinal"]
# df = pd.DataFrame(list(scFinal.find()))
# del df['_id']
app = FastAPI()

@app.get("/nb_article")
def getNbArticles():
    return {
            "nb_articles": df.shape[0],
            "nb_articles_acm": df[df.Publisher=="Science direct"].shape[0],
            "nb_articles_science": df[df.Publisher=="ACM"].shape[0],
            "nb_articles_ieee": df[df.Publisher=="IEEE"].shape[0]
           }

@app.get("/all_articles")
def getArticles():
    data=[]
    result=scFinal.find({})
    for i in result:
        data.append({
            "Titre":i['Document Title'],
            "year":i['Publication Year'],
            #"Authors":i['Authors'],
            "Citations":i['Citations'],
            "Afficliation":i['Affiliations'],
            "Publisher":i['Publisher'],
            "doi":i['DOI']
        })
    return data

@app.get("/countries")
def getCountCountries():
    data=[]
    grouped = df.groupby("Affiliations").count()
    # Iterate through the groups and counts
    for group, count in grouped["Citations"].items():
        data.append({
            'country':group,
            'count': count
        })
    return data
    
@app.get("/years")
def getCountYears():
    data=[]
    grouped = df.groupby("Publication Year").count()
    # Iterate through the groups and counts
    for group, count in grouped["Citations"].items():
        data.append({
            'Year':group,
            'count': count
        })
    return data
    
@app.get("/citations")
def getCountCitations():
    data=[]
    grouped = df.groupby("Affiliations")["Citations"].sum()
    # Iterate through the groups and counts
    for group, count in grouped.items():
        data.append({
            'Country':group,
            'Citations': count
        })
    return data


    
@app.get("/citationsCountries")
def getCountCitationsAndCountries():
    data=[]
    grouped1 = df.groupby("Affiliations").count()
    grouped = df.groupby("Affiliations")["Citations"].sum()
    # Iterate through the groups and counts
    for group1, group2 in zip(grouped.items(), grouped1["Publication Year"].items()):
        data.append({
            'Country':group1[0],
            'Citations': group1[1],
            'count':group2[1]
        })
    return data

@app.get("/citationsByPublisher")
def getCountCitationsByPublisherr():
    data=[]
    grouped = df.groupby("Publisher")["Citations"].sum()
    # Iterate through the groups and counts
    for group, count in grouped.items():
        data.append({
            'Publisher':group,
            'Citations': count
        })
    return data

In [14]:
pip install uvicorn

     -------------------------------------- 56.9/56.9 kB 249.4 kB/s eta 0:00:00
     ---------------------------------------- 96.6/96.6 kB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
import uvicorn

if __name__ == "__main__":
    uvicorn.run(app)


RuntimeError: asyncio.run() cannot be called from a running event loop

In [11]:
getNbArticles()

{'nb_articles': 2708,
 'nb_articles_acm': 1108,
 'nb_articles_science': 640,
 'nb_articles_ieee': 960}

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2708 entries, 0 to 2707
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Document Title    2708 non-null   object 
 1   Authors           2700 non-null   object 
 2   Universities      2708 non-null   object 
 3   Citations         2708 non-null   float64
 4   Affiliations      2708 non-null   object 
 5   Publisher         2708 non-null   object 
 6   DOI               2708 non-null   object 
 7   Publication Year  2708 non-null   int64  
dtypes: float64(1), int64(1), object(6)
memory usage: 169.4+ KB
